## Виды SQL запросов  
- Data Definition Language (DDL): создание/удаление таблиц, изменение структуры БД.  
- Data Manipulation Language (DML): добавление, изменение, извлечение данных из БД.  
- Data Control Language (DCL): управление правами доступа.
- Transaction Control Language (TCL): управление транзакциями*.

*Транзакция - логическая единица работы с данными. Цель существования - обеспечение целостности данных.   
В случае неудачи, все изменения в пределах транзакции "откатываются".

## sqlite3. Начало работы.
СУБД SQLite отличается от других СУБД возможностью её использования "из коробки". SQLite не требует отдельно запускаемый сервер, а сама БД сохраняется в файл, который затем можно использовать на любой платформе.  

Библиотека [sqlite3](https://docs.python.org/3/library/sqlite3.html) языка Python предлагает функционал для работы с SQLite.

In [ ]:
import sqlite3

In [ ]:
# Соединение (connection) с базой на диске
conn = sqlite3.connect("best_db_ever.db")
type(conn)

sqlite3.Connection

In [ ]:
# Курсор (cursor) используется для выполнения SQL-инструкций и получения результатов SQL-запросов
cur = conn.cursor()

In [ ]:
# Создадим таблицу movie. Указание типов данных для столбцов в SQLite не является обязательным
# Подробнее https://www.sqlite.org/flextypegood.html 
cur.execute("CREATE TABLE movie(title, year, score)")

Можно проверить, что таблица действительно создалась.  
SQLite имеет [встроенную таблицу](https://www.sqlite.org/schematab.html), хранящую схему всех имеющихся в БД таблиц

In [ ]:
res = cur.execute("SELECT name FROM sqlite_master")
res.fetchone()

('movie',)

Теперь добавим данные в созданную таблицу movie.

In [ ]:
cur.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")

Можно выполнить предыдущую ячейку несколько раз и посмотреть, к чему это приведет😏

In [ ]:
cur.execute("SELECT rowid, * FROM movie").fetchall()

[(1, 'Monty Python and the Holy Grail', 1975, 8.2),
 (2, 'And Now for Something Completely Different', 1971, 7.5)]

Закроем соединение с БД, откроем заново и проверим сохранность изменений

In [ ]:
conn.close()
conn = sqlite3.connect("best_db_ever.db")
cur = conn.cursor()

In [ ]:
cur.execute("SELECT * FROM movie").fetchall()

[('Monty Python and the Holy Grail', 1975, 8.2),
 ('And Now for Something Completely Different', 1971, 7.5)]

[По умолчанию](https://docs.python.org/3/library/sqlite3.html#sqlite3-controlling-transactions) выполнение запросов INSERT, UPDATE, DELETE и REPLACE *методами execute* и *executemany* - о котором речь впереди - открывает транзакцию, которую требуется закрыть вызовом *commit* или "откатить" вызовом *rollback*.  
Успешное завершение транзакции приведет к изменениям в БД.

In [ ]:
conn.commit()

Чтобы не забывать завершать транзакцию, можно пользоваться [менеджером контекста](https://docs.python.org/3/library/sqlite3.html#how-to-use-the-connection-context-manager) sqlite3.Connection и курсор тогда не понадобится.

In [ ]:
# Успешное выполнение запроса, con.commit() вызовется автоматически
with conn:
    conn.execute("""INSERT INTO movie VALUES
        ('Ирония судьбы, или С лёгким паром!', 1975, 8.2)
        """)
    
# В случае выброса исключения при выполнении запроса, вызовется con.rollback()
try:
    with conn:
        conn.execute("INSERT INTO movie(name) VALUES(?)", ("WALL-E",))
except sqlite3.OperationalError:
    print("Что-то пошло не так, вызов rollback()")

# Соединение должно быть закрыто "в ручную".
# При использовании менеджер контекста Соединения автоматически выполняется только завершение или откат транзакции.
conn.close()

Что-то пошло не так, вызов rollback()


Откроем соединение вновь и проверим изменения.

In [ ]:
conn = sqlite3.connect("best_db_ever.db")

In [ ]:
with conn:
  print(conn.execute('''SELECT title FROM movie 
                        WHERE score > 8.0
                     ''').fetchall())
  print(conn.execute('''SELECT * FROM movie
                        WHERE title = "WALL-E"
                     ''').fetchall())

[('Monty Python and the Holy Grail',), ('Ирония судьбы, или С лёгким паром!',)]
[]


Для запроса SELECT нет автоматического открытия транзакции. При выходе за пределы менеджера контекста завершение транзакции не вызовется.

### Подстановка значений и множественное выполнение запроса  
Подробнее о [безопасной подстановке](https://docs.python.org/3/library/sqlite3.html#sqlite3-placeholders).

In [ ]:
data = [
    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
    ("Monty Python's The Meaning of Life", 1983, 7.5),
    ("Monty Python's Life of Brian", 1979, 8.0),
]
with conn:
  conn.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)

In [ ]:
for row in conn.execute('''SELECT title, year FROM movie 
                        WHERE title LIKE 'Monty Python%'
                     '''):
  print('Найден:', row, sep='\t')

Найден:	('Monty Python and the Holy Grail', 1975)
Найден:	('Monty Python Live at the Hollywood Bowl', 1982)
Найден:	("Monty Python's The Meaning of Life", 1983)
Найден:	("Monty Python's Life of Brian", 1979)


А зачем тогда раньше Курсор создавали, если и так [работает](https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.execute)?

In [ ]:
conn.close()

## SQL-тренажер
Задачи на DML запросы разного уровня сложности с сылками на теоретические справки: https://sql-ex.ru/  
Требуется регистрация по почте.  


## ORM 
ORM (Object-Relational Mapping, объектно-реляционное отображение)  

Подход, позволяющий перейти от работы с реляционными БД как набором таблиц к оперированию объектами.  
Рассмотрим ORM-библиотеку [peewee](http://docs.peewee-orm.com/en/latest/) на языке Python. 

In [3]:
!pip3 install peewee

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 860 kB 5.2 MB/s 
  Created wheel for peewee: filename=peewee-3.15.4-cp38-cp38-linux_x86_64.whl size=701717 sha256=f300d6a1c6aad748e85f1d2e260f7eca4110d28ec7f828fb56534d580a811ba6
  Stored in directory: /root/.cache/pip/wheels/85/86/97/f8cb63cd8062bcb1de3973a30bb2629be75d97a413a620022f
Successfully built peewee


Для вдохновения был взят [проект](https://github.com/idemintsev/Weather-forecast).  
Исходные коды модифицированной версии, представленной в данном материале, расположены [здесь](https://github.com/ekuzina/Weather-forecast).

## database_manager.py

In [4]:
import peewee
from datetime import datetime

In [5]:
database = peewee.SqliteDatabase('/content/weather_forecast.db')

In [6]:
class BaseTable(peewee.Model):
    class Meta:
        database = database

class Location(BaseTable):
    # автоинкрементный первичный ключи формируется автоматически
    location  = peewee.CharField(unique = True)
    latitude  = peewee.DoubleField()
    longitude = peewee.DoubleField()

    # название таблицы формируется автоматически

class WeatherForecast(BaseTable):
    date = peewee.DateField(unique = True)
    weather = peewee.CharField()
    temperature_day   = peewee.SmallIntegerField()
    temperature_night = peewee.SmallIntegerField()
    
    location = peewee.ForeignKeyField(Location, to_field='id')

In [7]:
class DatabaseManager():
    #Выполняет запись и чтение из БД.

    def __init__(self):
        self.hardcode_location()

    def hardcode_location(self):
        result = (Location
          .insert(
            location = 'НИЯУ МИФИ (г. Москва)',
            latitude = 55.6501703776999,
            longitude = 37.66533398689662
          )
          .on_conflict_ignore()
          .execute())        
    
    # Извлечение местоположения из таблицы Location
    def get_location(self):
        query = Location.select().where(Location.id == 1).limit(1)
        for q in query:
            return q.location, q.latitude, q.longitude

    # Извлечение местоположения по вторичному ключу (в учебных целях)
    def get_location_fk(self):
        query = ( Location
         .select(Location.location.alias('loc'))
         .join(WeatherForecast)
         .where(WeatherForecast.location == 1).limit(1)
        )
        for q in query:
            return q.loc

    def save(self, data):
        result = (WeatherForecast
          .insert(
            date = data[0],
            weather = data[1][0],
            temperature_day = data[1][1],
            temperature_night = data[1][2],
            location = 1
          )
          .on_conflict_replace()
          .execute())

    # Проверка корректности формата даты
    @staticmethod
    def check_date_str(date):
        if date:
            try:
                date = datetime.strptime(date , "%Y-%m-%d").date()
            except ValueError as err:
                date = None
        return date

    def get_last_day(self):
        return WeatherForecast.select(peewee.fn.MAX(WeatherForecast.date)).scalar()
    
    # Формирование значений граничных дат по умолчанию
    def prepare_range(self, _from, _to):
        _from = DatabaseManager.check_date_str(_from)
        if not _from:
            today = datetime.today().date()
            _from = today

        _to = DatabaseManager.check_date_str(_to)
        if not _to:
            _to = self.get_last_day()

        if _from > _to:
            _from, _to = _to, _from

        return _from, _to


    def get(self, _from = None, _to = None):
        _from, _to = self.prepare_range(_from, _to)
        res = WeatherForecast.select().where(WeatherForecast.date.between(_from, _to))
        return res 

    def print(self, _from, _to=None):
        res = self.get(_from, _to)
        for data in res:
            print('{0}:\t{1}. Днем {2:+}*C, ночью {3:+}*C.'\
                .format(data.date.strftime('%Y-%m-%d'), data.weather, data.temperature_day, data.temperature_night))


## forecast_engine.py

In [8]:
import requests
from bs4 import BeautifulSoup

In [9]:
class WeatherMaker:

    def __init__(self, lat, lon):
        self.url = f'https://yandex.com.am/weather/?lat={lat}&lon={lon}'
        self.headers = {'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.114 YaBrowser/22.9.1.1110 (beta) Yowser/2.5 Safari/537.36'}
        self.forecast = {}

    # Первый этап парсинг данных с сайта
    @staticmethod
    def forecast_parse(web_response):
        weather_parser = BeautifulSoup(web_response, features='html.parser')

        list_of_days = weather_parser.find_all('time', {'class': 'time forecast-briefly__date'})

        day_temperature = weather_parser.find_all('div',
            {'class': 'temp forecast-briefly__temp forecast-briefly__temp_day'})
        
        night_temperature = weather_parser.find_all('div',
            {'class': 'temp forecast-briefly__temp forecast-briefly__temp_night'})
        
        atmospheric_precipitation = weather_parser.find_all('div', {'class': 'forecast-briefly__condition'})
        
        return list_of_days, atmospheric_precipitation, day_temperature, night_temperature

    # Второй этап парсинга (очистка полученных данных)
    @staticmethod
    def forecast_cleanup(day, atmospheric, day_temperature, night_temperature):
        day = WeatherMaker.date_cleanup(day)
        atmospheric = WeatherMaker.atmospheric_cleanup(atmospheric)
        day_temperature = WeatherMaker.temperature_cleanup(day_temperature)
        night_temperature = WeatherMaker.temperature_cleanup(night_temperature)
        return [day, atmospheric, day_temperature, night_temperature]
    
    @staticmethod
    def atmospheric_cleanup(atmospheric):
        return atmospheric.text

    @staticmethod
    def temperature_cleanup(temperature):
        temp = temperature.find('span', {'class': 'temp__value temp__value_with-unit'}).text
        # Замена символа для дальнейшего преобразования типов
        temp = str(temp).replace(chr(8722), '-')
        try:
            temp = int(temp)
        except Exception as e:
            temp = 10000
        return temp

    @staticmethod
    def date_cleanup(day):
        date_ = str(day.get('datetime'))[:10]
        date_ = datetime.strptime(date_, "%Y-%m-%d").date()
        return date_

    # Запись в словарь прогнозов {дата: [атм. осадки, дневная температура, ночная температура]}
    def read(self):
        web_response = requests.get(self.url, headers = self.headers)

        if web_response.status_code == 200:
            list_of_days, atmospheric_precipitation, day_temperature, night_temperature = \
                                                        WeatherMaker.forecast_parse(web_response.text)
        else:
            print('Bad response status code...')
            return
        
        for day, atmospheric, day_temperature, night_temperature in zip \
                    (list_of_days, atmospheric_precipitation, day_temperature, night_temperature):
            forecast_instance = WeatherMaker.forecast_cleanup(day, atmospheric, day_temperature, night_temperature)            
            self.forecast[forecast_instance[0]] = forecast_instance[1:]
        
    def get_last_day(self):
        return list(self.forecast.keys())[-1]       

    def get_first_day(self):
        return list(self.forecast.keys())[0]
    
    def get_location(self):
        return self.location

    # Проверка корректности формата даты
    @staticmethod
    def check_date_str(date): 
        if date:
            try:
                date = datetime.strptime(date, "%Y-%m-%d").date()
            except ValueError as err:
                date = None
        return date

    # Формирование значений граничных дат по умолчанию
    def prepare_range(self, _from, _to):
        _from = WeatherMaker.check_date_str(_from)
        if not _from:
            _from = self.get_first_day()
        
        _to = WeatherMaker.check_date_str(_to)
        if not _to:
            _to = self.get_last_day()

        if _from > _to:
            _from, _to = _to, _from            
        return _from, _to
 
    def get(self, _from = None, _to = None):
        _from, _to = self.prepare_range(_from, _to)            

        for _day in self.forecast.keys():
            if _from <= _day <= _to:
              yield (_day, self.forecast[_day])

## weather.py

In [14]:
import argparse
import time
from datetime import datetime, timedelta

import atexit

In [15]:
# Закрытие соединения с БД на выходе из приложения
@atexit.register
def goodbye():
    print('Closing DB...')
    database.close()

In [ ]:
if __name__ == '__main__':

    # database.connect()
    database.create_tables([WeatherForecast, Location])
    db_manager = DatabaseManager()

    loc, lat, lon = db_manager.get_location()
    weather = WeatherMaker(lat, lon)

    try:
        weather_console = argparse.ArgumentParser(description='Позволяет работать с прогнозом погоды')
        
        # Регистрация консольных аргументов
        weather_console.add_argument(
            '--get-loc', type=bool, dest='_loc', default=False,
            help='Местоположение по внешнему ключу.'
        )

        weather_console.add_argument(
            '--upd-from', type=str, dest='upd_from', default=None,
            help='Дата, с которой надо обновить/дополнить сохраненный в БД прогноз.'
                 'Формат 2020-12-31'
        )
        weather_console.add_argument(
            '--upd-to', type=str, default=None, dest='upd_to',
            help='Дата, до которой надо обновить/дополнить сохраненный в БД прогноз.'
                 'Формат 2020-12-31'
        )
        weather_console.add_argument(
            '--print-from', type=str, dest='print_from', default=None,
            help='Дата, начиная с которой надо вывести на консоль прогноз.'
                 'Формат 2020-12-31'
        )
        weather_console.add_argument(
            '--print-to', type=str, default=None, dest='print_to',
            help='Дата, до которой надо вывести на консоль прогноз.'
                 'Формат 2020-12-31'
        )

        weather_console.add_argument(
            '--show-prev', type=int, default=-1, dest='show_prev',
            help='Количество архивных дней, за которые нужно вывести на консоль прогноз.'
        )

        # Обработка переданных агрументов
        args = weather_console.parse_args(args = '--get-loc True'.split())

        # Логика согласно переданным аргументам
        
        if args._loc:
            print('В БД хранятся прогнозы для', db_manager.get_location_fk())

        if args.show_prev >= 0: 
            today = datetime.today()
            first_archived_day = (today - timedelta(days = args.show_prev)).strftime('%Y-%m-%d')
            today = today.strftime('%Y-%m-%d')

            print(f'\nПогода над {loc} в последние дни:')
            print('*' * 35)        
            db_manager.print(_from=first_archived_day, _to=today)
            print('*' * 35, '\n')        

        if args.upd_from or args.upd_to:
            weather.read()
            for val in weather.get(_from = args.upd_from, _to = args.upd_to):
                db_manager.save(data = val)
                # print(val)

        if args.print_from or args.print_to:
            print(f'\nПрогноз погоды над {loc}:')
            print('*' * 35)        
            db_manager.print(args.print_from, args.print_to)
            print('*' * 35, '\n')        

    except Exception as exc:
        print(f'Что-то пошло не так: {exc}')